# 从零开始llama3

In [ ]:
from pathlib import Path
import tiktoken
from tiktoken.load import load_tiktoken_bpe
import torch
import json
%matplotlib inline
import matplotlib.pyplot as plt




UsageError: Line magic function `%` not found.


In [ ]:
#SDK download model
# from modelscope import snapshot_download
# model_dir = snapshot_download('wdndev/Meta-Llama-3-8B-Instruct-2layers')

## tokenizer
使用字节对编码（Byte Pair Encoding, BPE）作为分词器

andrej karpathy 实现BPE tokenizer 的链接：

创建分词器实例`tiktoken.Encoding`，其中主要设置文本拆分规则`pat_str`，合并规则`mergeable_ranks`，特殊的token`special_tokens`。

正则表达式，
1. `(?i:'s|'t|'re|'ve|'m|'ll|'d)`中`?i:`表示忽略大小写，其中这些缩写单独作为一个token；
2. `[^\r\n\p{L}\p{N}]?\p{L}+`：匹配既不是换行符、字母，也不是数字的字符（例如标点符号），后面跟着一个或多个字母；
3. `\p{N}{1,3}`匹配1到3位数字；
4. `?[^\s\p{L}\p{N}]+[\r\n]*`一个可选的空格，后面跟着一个或多个标点符号，再后面跟着 0 个或多个换行符；
5. `\s*[\r\n]+`匹配0 个或多个空白字符，后面跟着 1 个或多个换行符；
6. `\s+(?!\S)`匹配1 个或多个空白字符，且这些空白字符后面不能有非空白字符，例如文本末尾的空格；
7. `\s+`匹配 1 个或多个空白字符。

In [ ]:
# 加载训练好的分词器
tokenizer_path = "wdndev/Meta-Llama-3-8B-Instruct-2layers/tokenizer.model"
mergeable_ranks = load_tiktoken_bpe(tokenizer_path)
# 设置特殊token
special_tokens = [
            "<|begin_of_text|>",
            "<|end_of_text|>",
            "<|reserved_special_token_0|>",
            "<|reserved_special_token_1|>",
            "<|reserved_special_token_2|>",
            "<|reserved_special_token_3|>",
            "<|start_header_id|>",
            "<|end_header_id|>",
            "<|reserved_special_token_4|>",
            "<|eot_id|>",  # end of turn
        ] + [f"<|reserved_special_token_{i}|>" for i in range(5, 256 - 5)]
# 创建token编码器
tokenizer = tiktoken.Encoding(
    name=Path(tokenizer_path).name,
    pat_str=r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+",
    mergeable_ranks=mergeable_ranks,
    special_tokens={token: len(mergeable_ranks) + i for i, token in enumerate(special_tokens)},
)
# 测试编码解码功能
tokenizer.encode("hello world!"),tokenizer.decode(tokenizer.encode("hello world!"))

([15339, 1917, 0], 'hello world!')

## 读取模型文件

因原版 Llama3 8B 模型32层 Transformers，如果加载全部的参数，16G内存机器加载失败，故选取原版 Llama3 8B 模型权重的前2层，重新保存，大小约为2.7G。

In [ ]:
# 加载模型权重
# 这里使用的是减少层数的参数文件，仅用于学习模型处理过程
model = torch.load("wdndev\Meta-Llama-3-8B-Instruct-2layers\consolidated_2layers.pth")
print(json.dumps(list(model.keys())[:20], indent=4))

C:\Users\86984\AppData\Local\Temp\ipykernel_23140\2382376065.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("wdndev\Meta-Llama-3-8B-Instruct-2layers\

[
    "tok_embeddings.weight",
    "layers.0.attention.wq.weight",
    "layers.0.attention.wk.weight",
    "layers.0.attention.wv.weight",
    "layers.0.attention.wo.weight",
    "layers.0.feed_forward.w1.weight",
    "layers.0.feed_forward.w3.weight",
    "layers.0.feed_forward.w2.weight",
    "layers.0.attention_norm.weight",
    "layers.0.ffn_norm.weight",
    "layers.1.attention.wq.weight",
    "layers.1.attention.wk.weight",
    "layers.1.attention.wv.weight",
    "layers.1.attention.wo.weight",
    "layers.1.feed_forward.w1.weight",
    "layers.1.feed_forward.w3.weight",
    "layers.1.feed_forward.w2.weight",
    "layers.1.attention_norm.weight",
    "layers.1.ffn_norm.weight",
    "norm.weight"
]


In [ ]:
# 获取模型配置参数
with open("wdndev\Meta-Llama-3-8B-Instruct-2layers\params.json", "r") as f:
    config = json.load(f)
config

{'dim': 4096,
 'n_layers': 2,
 'n_heads': 32,
 'n_kv_heads': 8,
 'vocab_size': 128256,
 'multiple_of': 1024,
 'ffn_dim_multiplier': 1.3,
 'norm_eps': 1e-05,
 'rope_theta': 500000.0}

In [ ]:
# 从配置文件中提取模型参数
dim = config["dim"]
n_layers = config["n_layers"]
n_heads = config["n_heads"]
n_kv_heads = config["n_kv_heads"]
vocab_size = config["vocab_size"]
multiple_of = config["multiple_of"]
ffn_dim_multiplier = config["ffn_dim_multiplier"]
norm_eps = config["norm_eps"]
rope_theta = torch.tensor(config["rope_theta"])

## 将文本转换为token
这里使用 tiktoken（OpenAI 的库）作为分词器

In [ ]:
prompt = "the answer to the ultimate question of life, the universe, and everything is "

# 编码为token
# <|begin_of_text|>的token_id=128000
tokens = [128000] + tokenizer.encode(prompt)
print(tokens)
tokens = torch.tensor(tokens)

# 将每个 token 解码为对应的文本
prompt_split_as_tokens = [tokenizer.decode([token.item()]) for token in tokens]
print(prompt_split_as_tokens)

[128000, 1820, 4320, 311, 279, 17139, 3488, 315, 2324, 11, 279, 15861, 11, 323, 4395, 374, 220]
['<|begin_of_text|>', 'the', ' answer', ' to', ' the', ' ultimate', ' question', ' of', ' life', ',', ' the', ' universe', ',', ' and', ' everything', ' is', ' ']


## 将token转换为embedding
这里使用内置的神经网络模块

无论如何, [17x1] token 现在是 [17x4096]，即每个 token 的长度为 4096 的 embeddings

注意：跟踪 shapes，这样一切将变得理解更容易

In [ ]:
# 加载嵌入层并复制权重
embedding_layer = torch.nn.Embedding(vocab_size, dim)
embedding_layer.weight.data.copy_(model["tok_embeddings.weight"])

# 获取未归一化的 token 嵌入
token_embeddings_unnormalized = embedding_layer(tokens).to(torch.bfloat16)
token_embeddings_unnormalized.shape

torch.Size([17, 4096])

## 构建RMS 归一化嵌入
请注意，经过此步骤后 shapes 不变， 只是值被归一化

需要注意的是，需要一个 norm_eps（来自配置）以避免不小心将 RMS 设置为 0 并导致除以 0 的情况

公式如下：


In [ ]:
# rms 归一化函数

# def rms_norm(tensor, norm_weights):
#     rms = (tensor.pow(2).mean(-1, keepdim=True) + norm_eps)**0.5
#     return tensor * (norm_weights / rms)

def rms_norm(tensor, norm_weights):
    return (tensor * torch.rsqrt(tensor.pow(2).mean(-1, keepdim=True) + norm_eps)) * norm_weights

## 构建第一个Transformer层

### 1.归一化
从模型字典中访问 layer.0 （这是第一层）

embedding input -> RMS_Norm

In [ ]:
# 归一化token嵌入
token_embeddings = rms_norm(token_embeddings_unnormalized, model["layers.0.attention_norm.weight"])
token_embeddings.shape

torch.Size([17, 4096])

### 2.从头实现注意力机制
加载第一个 Transformer 层的注意力头

当我们从模型中加载 query， key，value 和 output 向量时，注意到 shapes 分别为 [4096x4096]， [1024x4096]， [1024x4096]， [4096x4096]

乍一看这有些奇怪，因为在理想情况下我们希望每个头单独拥有各自的 q，k，v 和 o

这里作者将其捆绑在一起，为什么会这样呢? 因为这样有助于并行化注意力头的计算

将展开所有内容...

In [ ]:
# 打印第一个层的注意力权重 shapes
print(
    model["layers.0.attention.wq.weight"].shape,
    model["layers.0.attention.wk.weight"].shape,
    model["layers.0.attention.wv.weight"].shape,
    model["layers.0.attention.wo.weight"].shape
)

torch.Size([4096, 4096]) torch.Size([1024, 4096]) torch.Size([1024, 4096]) torch.Size([4096, 4096])


### 展开query
在下一节中，将展开多个注意力头的 query，得到的 shapes 为 [32x128x4096]

这里的 32 是 Llama3 的注意力头数量，128 是 query 向量的大小，4096 是 token 嵌入的大小

In [ ]:
# reshape query 权重为[头数，头维度，嵌入维度]

q_layer0 = model["layers.0.attention.wq.weight"]
head_dim = q_layer0.shape[0] // n_heads
q_layer0 = q_layer0.view(n_heads, head_dim, dim)
q_layer0.shape

torch.Size([32, 128, 4096])

### 实现第一层的第一个头
这里查询了第一个层的第一个头的 query 权重矩阵，其大小为 [128x4096]

In [ ]:
q_layer0_head0 = q_layer0[0]
q_layer0_head0.shape

torch.Size([128, 4096])

### 现在将 query 权重与 token 嵌入相乘，以获得每个 token 的 query
这里可以看到得到的 shape 是 [17x128]， 这是因为有 17 个 token，每个 token 有一个长度为 128 的 query

In [ ]:
q_per_token = torch.matmul(token_embeddings, q_layer0_head0.T)
q_per_token.shape

torch.Size([17, 128])

### 位置编码
当前，每个 token 都有一个 query 向量，但如果你想一想 -- 其实各个 query 向量并不知道它们在 prompt 中的位置。

In [ ]:
# 使用复数点积计算旋转向量
zero_to_one_split_into_64_parts = torch.tensor(range(64))/64
zero_to_one_split_into_64_parts
freqs = 1.0 / (rope_theta ** zero_to_one_split_into_64_parts)
freqs_for_each_token = torch.outer(torch.arange(17), freqs)
freqs_cis = torch.polar(torch.ones_like(freqs_for_each_token), freqs_for_each_token)

In [ ]:
q_per_token_split_into_pairs = q_per_token.float().view(q_per_token.shape[0], -1, 2)
print("q_per_token_split_into_pairs.shape",q_per_token_split_into_pairs.shape)

q_per_token_as_complex_numbers = torch.view_as_complex(q_per_token_split_into_pairs)
print("q_per_token_as_complex_numbers.shape",q_per_token_as_complex_numbers.shape)

q_per_token_as_complex_numbers_rotated = q_per_token_as_complex_numbers * freqs_cis
print("q_per_token_as_complex_numbers_rotated.shape", q_per_token_as_complex_numbers_rotated.shape)

q_per_token_split_into_pairs_rotated = torch.view_as_real(q_per_token_as_complex_numbers_rotated)
print("q_per_token_split_into_pairs_rotated.shape", q_per_token_split_into_pairs_rotated.shape)

q_per_token_rotated = q_per_token_split_into_pairs_rotated.view(q_per_token.shape)
print("q_per_token_rotated.shape", q_per_token_rotated.shape)

q_per_token_split_into_pairs.shape torch.Size([17, 64, 2])
q_per_token_as_complex_numbers.shape torch.Size([17, 64])
q_per_token_as_complex_numbers_rotated.shape torch.Size([17, 64])
q_per_token_split_into_pairs_rotated.shape torch.Size([17, 64, 2])
q_per_token_rotated.shape torch.Size([17, 128])


In [ ]:
# key
k_layer0 = model["layers.0.attention.wk.weight"]
k_layer0 = k_layer0.view(n_kv_heads, k_layer0.shape[0] // n_kv_heads, dim)
print("k_layer0.shape", k_layer0.shape)

k_layer0_head0 = k_layer0[0]
print("k_layer0_head0.shape", k_layer0_head0.shape)

k_per_token = torch.matmul(token_embeddings, k_layer0_head0.T)
print("k_per_token.shape", k_per_token.shape)

k_per_token_split_into_pairs = k_per_token.float().view(k_per_token.shape[0], -1, 2)
print("k_per_token_split_into_pairs.shape",k_per_token_split_into_pairs.shape)

k_per_token_as_complex_numbers = torch.view_as_complex(k_per_token_split_into_pairs)
print("k_per_token_as_complex_numbers.shape", k_per_token_as_complex_numbers.shape)

k_per_token_split_into_pairs_rotated = torch.view_as_real(k_per_token_as_complex_numbers * freqs_cis)
print("k_per_token_split_into_pairs_rotated.shape", k_per_token_split_into_pairs_rotated.shape)

k_per_token_rotated = k_per_token_split_into_pairs_rotated.view(k_per_token.shape)
print("k_per_token_rotated.shape", k_per_token_rotated.shape)


k_layer0.shape torch.Size([8, 128, 4096])
k_layer0_head0.shape torch.Size([128, 4096])
k_per_token.shape torch.Size([17, 128])
k_per_token_split_into_pairs.shape torch.Size([17, 64, 2])
k_per_token_as_complex_numbers.shape torch.Size([17, 64])
k_per_token_split_into_pairs_rotated.shape torch.Size([17, 64, 2])
k_per_token_rotated.shape torch.Size([17, 128])


### 接下来，将 query 和 key 的矩阵相乘

In [ ]:
qk_per_token = torch.matmul(q_per_token_rotated, k_per_token_rotated.T)/(head_dim)**0.5
qk_per_token.shape

torch.Size([17, 17])

### 屏蔽QK分数
在 llama3 的训练过程中，未来的 token qk 分数被屏蔽。

为什么？因为在训练过程中，只学习使用过去的 token 来预测 token 。

因此，在推理过程中，将未来的 token 设置为零。

In [ ]:
def display_qk_heatmap(qk_per_token):
    _, ax = plt.subplots()
    im = ax.imshow(qk_per_token.to(float).detach(), cmap='viridis')
    ax.set_xticks(range(len(prompt_split_as_tokens)))
    ax.set_yticks(range(len(prompt_split_as_tokens)))
    ax.set_xticklabels(prompt_split_as_tokens)
    ax.set_yticklabels(prompt_split_as_tokens)
    ax.figure.colorbar(im, ax=ax)
    
# display_qk_heatmap(qk_per_token)

: 